# PySpark - Preparing the Data For Modeling

## Introduction

This project aims to explore the methods for preparing the dataset for modeling purposes. It is important to note that any dataset is dirty until proven otherwise and that it should be proven to be sufficiently clean before using it. However, no dataset can be entirely clean. Below will list some of the problems that can occur in a dataset. Majority of the time, 80% of the work is getting familiar and cleaning up the dataset. The remaining 20% would be building the model.

For this project, the dataset used will only consist of 22 records, as this is to get a feel for data cleaning with PySpark and should be transferable to other datasets.

## Problems that a Dataset can have:
- __Duplicated Observations__: These types of duplication comes from systemic and operator's faults.
- __Missing Observations__: These types of errors can come about due to sensor problems, data corruption or unwilling participant that would not provide answers.
- __Anomalous Observations__: Observations that stands out when compared to the rest of the dataset. Like Outliers.
- __Encoding__: This is when text fields are not normalised, in different languages, gibberish text inputs, or when date and date time fields were not encoded similarly.
- __Untrustworthy answers__: These are true when it comes to surveys. When the response is a lie for any number of reasons. This type is much harder to work with and clean up.


## Breakdown of this Notebook

- Handling Duplicates in data records
- Handling missing observations in dataset
- Handling outliers
- Exploring the descriptive statistics
- Computing Correlations
- Drawing Histograms to describe the data
- Visualising the interactions between features


## 1 PySpark Machine Configuration:

Here it only uses two processing cores from the CPU, and it set up by the following code.

In [1]:
%%configure
{
    "executorCores" : 4
}

In [2]:
from pyspark.sql.types import *

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 2 Setup the Correct Directory:

In [3]:
import os

# Change the Path:
path = '++++your working directory here++++/Datasets/'
os.chdir(path)
folder_pathway = os.getcwd()

# print(folder_pathway)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3 Create the Dataset: 22 samples

In [4]:
# Define the Dirty Dataset:
dirty_data = spark.createDataFrame(
    [(1,'Porsche','Boxster S','Turbo',2.5,4,22,None),
     (2,'Aston Martin','Vanquish','Aspirated',6.0,12,16,None),
     (3,'Porsche','911 Carrera 4S Cabriolet','Turbo',3.0,6,24,None),
     (3,'General Motors','SPARK ACTIV','Aspirated',1.4,None,32,None),
     (5,'BMW','COOPER S HARDTOP 2 DOOR','Turbo',2.0,4,26,None),
     (6,'BMW','330i','Turbo',2.0,None,27,None),
     (7,'BMW','440i Coupe','Turbo',3.0,6,23,None),
     (8,'BMW','440i Coupe','Turbo',3.0,6,23,None),
     (9,'Mercedes-Benz',None,None,None,None,27,None),
     (10,'Mercedes-Benz','CLS 550','Turbo',4.7,8,21,79231),
     (11,'Volkswagen','GTI','Turbo',2.0,4,None,None),
     (12,'Ford Motor Company','FUSION AWD','Turbo',2.7,6,20,None),
     (13,'Nissan','Q50 AWD RED SPORT','Turbo',3.0,6,22,None),
     (14,'Nissan','Q70 AWD','Aspirated',5.6,8,18,None),
     (15,'Kia','Stinger RWD','Turbo',2.0,4,25,None),
     (16,'Toyota','CAMRY HYBRID LE','Aspirated',2.5,4,46,None),
     (16,'Toyota','CAMRY HYBRID LE','Aspirated',2.5,4,46,None),
     (18,'FCA US LLC','300','Aspirated',3.6,6,23,None),
     (19,'Hyundai','G80 AWD','Turbo',3.3,6,20,None),
     (20,'Hyundai','G80 AWD','Turbo',3.3,6,20,None),
     (21,'BMW','X5 M','Turbo',4.4,8,18,121231),
     (22,'GE','K1500 SUBURBAN 4WD','Aspirated',5.3,8,18,None) ],
    schema = ['Id','Manufacturer','Model','EngineType','Displacement',
     'Cylinders','FuelEconomy','MSRP'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Inspect:
dirty_data.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Id=1, Manufacturer='Porsche', Model='Boxster S', EngineType='Turbo', Displacement=2.5, Cylinders=4, FuelEconomy=22, MSRP=None)]

## 4 Handling duplicates of data records:

It can be very hard to spot duplicates of data and these happen all the time. PySpark DataFrame have a method to help remove these duplicates called .dropDuplicates() transformation function.

In [6]:
# First is to check for duplicated rows:
dirty_data.count(), dirty_data.distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(22, 21)

#### From this "(22, 21)" output, it can be determined that there is one record of data that has a duplicate.

#### To check which record it is:
- First, use the .groupBy() function to define which of the dataset columns to aggregate. Here all the columns were chosen.
- Next, count the number of times these records occur with the .count() function.
- Next, use the .filter() method to select all of the rows in the dataset that occurs more than once.
- Lastly, print these records out with the .show() function.

In [7]:
# Inspect the dataset for duplicates:
(
    dirty_data
    .groupBy(dirty_data.columns)
    .count()
    .filter('count > 1')
    .show()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------+---------------+----------+------------+---------+-----------+----+-----+
| Id|Manufacturer|          Model|EngineType|Displacement|Cylinders|FuelEconomy|MSRP|count|
+---+------------+---------------+----------+------------+---------+-----------+----+-----+
| 16|      Toyota|CAMRY HYBRID LE| Aspirated|         2.5|        4|         46|null|    2|
+---+------------+---------------+----------+------------+---------+-----------+----+-----+

It can be seen that __"Id 16"__ is the duplicate record.

#### Next is to proceed in removing the duplicate row:

In [8]:
# Remove the duplicates:
fully_removed_dat = dirty_data.dropDuplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 4.1 Duplicates: data IDs

If the data is collected over time, it can be possible that the same data would be recorded with diffrent IDs. 

#### To check for Duplicate IDs:
- First groupBy all the columns except for the "Id" column.
- Next, is to count the number of records.
- Next, is to extract the records that has a duplicate count. ("count > 1")
- Finally, is to print out the data.

In [9]:
# Inspect if the Dataset has duplicate IDs:
(
    fully_removed_dat
    .groupBy( [col for col in fully_removed_dat.columns if col != 'Id'] )
    .count()
    .filter('count > 1')
    .show()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------+----------+------------+---------+-----------+----+-----+
|Manufacturer|     Model|EngineType|Displacement|Cylinders|FuelEconomy|MSRP|count|
+------------+----------+----------+------------+---------+-----------+----+-----+
|         BMW|440i Coupe|     Turbo|         3.0|        6|         23|null|    2|
|     Hyundai|   G80 AWD|     Turbo|         3.3|        6|         20|null|    2|
+------------+----------+----------+------------+---------+-----------+----+-----+

#### Check the count similar to the previous section:

In [10]:
# Save the data as a separate copy:
no_ids_dat = (
    fully_removed_dat
    .select( [col for col in fully_removed_dat.columns if col != "Id"] )
)

# Compare the count:
no_ids_dat.count(), no_ids_dat.distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(21, 19)

#### From the output "(21, 19)", it shows that there are 4 duplicate records (or 2 duplicate IDs) in the dataset.

In [11]:
# Remove these duplicates:
id_removed_dat = fully_removed_dat.dropDuplicates(
    subset = [col for col in fully_removed_dat.columns if col != "Id"]
)

# Count the number of rows:
id_removed_dat.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

19

## 4.2 Duplicates: ID Collisions.

At this point, after removing duplicates from the previous section, there may still be remaining duplicate IDs that are collisions.

These may arise for multiple reasons such as:
- Instrumental error
- Insufficient data structure to store IDs data
- IDs representing some hash function of the record elements
- Arising collisions due to choice of hash function.

### Import the required Library:

In [12]:
import pyspark.sql.functions as f

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Check if the dataset has Collisions:

Rather than segmenting the operations as in the previous section, here, the code will execute all the required operations in one go.

To do this:
- Use the .count() method to count all the records that are of non-null values in a specified column.
- Next, use the .countDistinct() method to return a distinct count of the vlaues in the specifed column. 
- Alias both with different names for the columns.

In [13]:
id_removed_dat.agg(
    f.count('Id').alias('CountOfIDs'),
    f.countDistinct('Id').alias('CountOfDistinctIDs')
).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+
|CountOfIDs|CountOfDistinctIDs|
+----------+------------------+
|        19|                18|
+----------+------------------+

#### Here, it can be seen that there are 2 records with the same IDs.

### Check which IDs are duplicated in the Dataset:

In [14]:
(
    id_removed_dat
    .groupBy('Id')
    .count()
    .filter('count > 1')
    .show()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-----+
| Id|count|
+---+-----+
|  3|    2|
+---+-----+

#### Here it can be seen that ID 3 has duplicates.

## Examine the row that is ID 3:

In [15]:
(
    id_removed_dat
    .filter('Id == 3')
    .show()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------+--------------------+----------+------------+---------+-----------+----+
| Id|  Manufacturer|               Model|EngineType|Displacement|Cylinders|FuelEconomy|MSRP|
+---+--------------+--------------------+----------+------------+---------+-----------+----+
|  3|General Motors|         SPARK ACTIV| Aspirated|         1.4|     null|         32|null|
|  3|       Porsche|911 Carrera 4S Ca...|     Turbo|         3.0|        6|         24|null|
+---+--------------+--------------------+----------+------------+---------+-----------+----+

#### Here the records are not the same, however, they do share the same ID number (= 3).  To fix this, a new ID should be created that will be unique.

### To do this:
There is a PySpark SQL function called .monotonically_increasing_id() that will create a unique stream of IDs. 

Where it will:
- Create an alias of the ID column 

NOTE: this method is good for dataset that are less than 1 billion partitions and the records have to be less than 8 billion recrods in each of the partitions.

In [16]:
# Apply the fix:
new_id_dat = (
    id_removed_dat
    .select(
        [f.monotonically_increasing_id().alias('Id')] + 
        [col for col in id_removed_dat.columns if col != "Id"]
    )
)

# Inspect:
new_id_dat.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------------------+--------------------+----------+------------+---------+-----------+------+
|           Id|      Manufacturer|               Model|EngineType|Displacement|Cylinders|FuelEconomy|  MSRP|
+-------------+------------------+--------------------+----------+------------+---------+-----------+------+
|   8589934592|    General Motors|         SPARK ACTIV| Aspirated|         1.4|     null|         32|  null|
| 188978561024|     Mercedes-Benz|             CLS 550|     Turbo|         4.7|        8|         21| 79231|
| 197568495616|     Mercedes-Benz|                null|      null|        null|     null|         27|  null|
| 206158430208|Ford Motor Company|          FUSION AWD|     Turbo|         2.7|        6|         20|  null|
| 438086664192|               BMW|COOPER S HARDTOP ...|     Turbo|         2.0|        4|         26|  null|
| 523986010112|      Aston Martin|            Vanquish| Aspirated|         6.0|       12|         16|  null|
| 721554505728|    

## 5 Handling Missing data observations:

Missing data can be very common amongst datasets. Here, the dataset used will be "new_id_dat" from the previous section.

## 5.1 Check the percentage of the data that is Missing:

Each row and column will be checked for missing records. From these missing values, it will be determined what data is kept and what will be dropped, or potentially impute.

### Calc the number of missing observations in the ROWS:

To do this:
- It is easier to work with RDDs and loop through each of the element to count the missing values.
- First, access the RDD of the new_id_dat DataFrame and proceed with the .map() function.
- In the .map() function, loop through each row to extract the 'Id' and count the missing values with "sum([c == None for c in row])"
- The output of the .map() will be: Id, count of missing values.
- Next, select mssing values that has a count of more than 1 with .filter() function.
- Next, .collect() these records.
- Lastly, create a simple DataFrame using the .orderBy() function to show the records that have missing values and in descending order.

In [23]:
(
    spark.createDataFrame(
        new_id_dat
        .rdd
        .map(
            lambda row: (row['Id'], sum([c == None for c in row]))
        )
        .filter(lambda el: el[1] > 1)
        .collect()
        , ['Id', 'CountMissing']
    )
    .orderBy('CountMissing', ascending=False)
    .show()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+
|          Id|CountMissing|
+------------+------------+
|197568495616|           5|
|  8589934592|           2|
|919123001344|           2|
|721554505728|           2|
+------------+------------+

In [24]:
# Inspect the records: that have the highest missing values.
(
    new_id_dat
    .where('Id == 197568495616')
    .show()
)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------+-----+----------+------------+---------+-----------+----+
|          Id| Manufacturer|Model|EngineType|Displacement|Cylinders|FuelEconomy|MSRP|
+------------+-------------+-----+----------+------------+---------+-----------+----+
|197568495616|Mercedes-Benz| null|      null|        null|     null|         27|null|
+------------+-------------+-----+----------+------------+---------+-----------+----+

#### From the above, it can be seen that Mercedes-Benz consists of a lot of Null values.

### So, the next step:

Would be to remove this record entirely. 

Note: The 'thresh = 4' is set to only remove records that consists of a minimum of 4 non-missing values. This particular record only has 3 useful pieces of information.

In [25]:
merc_out = new_id_dat.dropna(thresh = 4)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# Confirm by counts:
new_id_dat.count(), merc_out.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(19, 18)

In [27]:
# Inspect if it was really removed: should be a blank DataFrame.
(
    merc_out
    .where('Id == 197568495616')
    .show()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------+-----+----------+------------+---------+-----------+----+
| Id|Manufacturer|Model|EngineType|Displacement|Cylinders|FuelEconomy|MSRP|
+---+------------+-----+----------+------------+---------+-----------+----+
+---+------------+-----+----------+------------+---------+-----------+----+

### Calc the number of missing observations in the Columns:

Here, the idea is to check if there are columns with low incidence of relevant information.

To do this:
- First, for the inner list that is "(1 - (f.count(c) / f.count('*'))).alias(c + '_miss')" and "for c in merc_out.columns", this is where a loop is used to go through all the columns in the merc_out DF, followed by a count on the number of non-missing values to be found in each of the columns. Next, is dividing it by the total count of all the rows. Then followed by subtracting this from 1, to get the percentage of missing values.

   func can be seen as -> ( *[ (1 - (f.count(c) / f.count('*'))).alias(c + '_miss') for c in merc_out.columns] )
   
- Note, that no calc. is done yet as Python only stores the information as lists of objects/pointers. When the .agg() function is called, then it gets translated into PySpark's internal execution graph, and only gets executed once the .collect() action is called and executed.
- The .collect() function will return a list of one element, that is a Row() object with aggregated information.
- These Row() objects are then converted in dictionaries by using the .asDict() first before the items can be extracted by using the .items() functions. The results from this will be a list of tuples. 
- This list of tuples will have two elements, where the first one is the "column name" which is appened with "_miss" by use of the .alias() function. The second element is the percentage of the missing observations.


In [29]:
for k, v in sorted(
    merc_out
    .agg(*[
        (1 - (f.count(c) / f.count('*'))).alias(c + '_miss')
        for c in merc_out.columns
    ])
    .collect()[0]
    .asDict()
    .items()
    , key = lambda el: el[1]
    , reverse = True
):
    print(k, v)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

MSRP_miss 0.8888888888888888
Cylinders_miss 0.11111111111111116
FuelEconomy_miss 0.05555555555555558
Id_miss 0.0
Manufacturer_miss 0.0
Model_miss 0.0
EngineType_miss 0.0
Displacement_miss 0.0

In [31]:
# Another way to represent the above is:

data = merc_out.agg(*[(1 - (f.count(c) / f.count('*'))).alias(c + '_miss') for c in merc_out.columns]).collect()[0].asDict().items()

for k, v in sorted( data, key = lambda el: el[1], reverse = True ):
    print(k, v)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

MSRP_miss 0.8888888888888888
Cylinders_miss 0.11111111111111116
FuelEconomy_miss 0.05555555555555558
Id_miss 0.0
Manufacturer_miss 0.0
Model_miss 0.0
EngineType_miss 0.0
Displacement_miss 0.0

#### From the above, it can be seen that the MSRP column has a lot of missing values (at 88%). Therefore, this column can be dropped, as it provides no useful information.

In [32]:
# Drop the column "MSRP":
no_MSRP = merc_out.select( [col for col in new_id_dat.columns if col != 'MSRP'] )

# Inspect:
no_MSRP.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------------------+--------------------+----------+------------+---------+-----------+
|           Id|      Manufacturer|               Model|EngineType|Displacement|Cylinders|FuelEconomy|
+-------------+------------------+--------------------+----------+------------+---------+-----------+
|   8589934592|    General Motors|         SPARK ACTIV| Aspirated|         1.4|     null|         32|
| 188978561024|     Mercedes-Benz|             CLS 550|     Turbo|         4.7|        8|         21|
| 206158430208|Ford Motor Company|          FUSION AWD|     Turbo|         2.7|        6|         20|
| 438086664192|               BMW|COOPER S HARDTOP ...|     Turbo|         2.0|        4|         26|
| 523986010112|      Aston Martin|            Vanquish| Aspirated|         6.0|       12|         16|
| 721554505728|        Volkswagen|                 GTI|     Turbo|         2.0|        4|       null|
| 764504178688|               Kia|         Stinger RWD|     Turbo|         2.0|   

## 5.2 Imputing the missing observations:

This means the replacement of missing data with substituted values.

There are __two__ ways to solve this:
- One, Pass a value that every "Null" or "None" in the dataset will be replaced with.
- Two, Pass a dictionary with different values in it for each of the column with missing records.

In this example, the 2nd approach will be used. 

#### To do this: Specify a Ratio between the Fuel Economy and Displacement | Number of cyclinders and displacement.

The formulas used to do these are the following:

- To replace the missing values in the fuel economy -> 
    fuel_economy_mult = fuel_economy / (displacement * cylinders)
    
- To replace the missing calues in number of cylinders -> 
    cylinders_mult = cylinders / displacement

These multipliers will be applied onto each row and taken as the average of these.
    
NOTE: these are not going to be accurate but serves the purpose. 

Once the formulas are applied, a dictionary can be created in Spark DataFrame is to use the .toPandss() function to convert the Spark DF to Pandas DF. This is then followed by a .to_dict() function to convert the data to a dictionary as proposed as the 2nd approach. 

The paramter that is "records" in the .to_dict() function, willcovert each row to a dictionary, where the key is the column name with the corresponding record value.

In [33]:
# Applying the formulas and create a Dictionary:

multipliers_dict = (
    no_MSRP
    .agg(
        f.mean(
            f.col('FuelEconomy') / (f.col('Displacement') * f.col('Cylinders'))
        ).alias('FuelEconomy'),
        f.mean(
            f.col('Cylinders') / f.col('Displacement')
        ).alias('Cylinders')
    )
).toPandas().to_dict('records')[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# Inspect:
multipliers_dict

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'FuelEconomy': 1.4957485048359973, 'Cylinders': 1.8353365984789105}

### Impute the Dataset with the created Dictionary above:

To do this:
- First, convert the original data to reflect the ratios specified earlier.
- Apply the multipliers_dict to fill in the missing values in the data.
- Lastly, revert the columns back to their orignal state.

The .withColumn() function overwrites the origninal column names.

Source: https://sparkbyexamples.com/spark/spark-dataframe-withcolumn/

In [42]:
imputed_dat = (
    no_MSRP
    .withColumn('FuelEconomy', f.col('FuelEconomy') / f.col('Displacement') / f.col('Cylinders'))
    .withColumn('Cylinders', f.col('Cylinders') / f.col('Displacement'))
    .fillna(multipliers_dict)
    .withColumn('Cylinders', (f.col('Cylinders') * f.col('Displacement')).cast('integer') )
    .withColumn('FuelEconomy', f.col('FuelEconomy') * f.col('Displacement') * f.col('Cylinders'))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
# Inspect the DF:
imputed_dat.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------------------+--------------------+----------+------------+---------+------------------+
|           Id|      Manufacturer|               Model|EngineType|Displacement|Cylinders|       FuelEconomy|
+-------------+------------------+--------------------+----------+------------+---------+------------------+
|   8589934592|    General Motors|         SPARK ACTIV| Aspirated|         1.4|        2| 4.188095813540793|
| 188978561024|     Mercedes-Benz|             CLS 550|     Turbo|         4.7|        8|              21.0|
| 206158430208|Ford Motor Company|          FUSION AWD|     Turbo|         2.7|        5|16.666666666666668|
| 438086664192|               BMW|COOPER S HARDTOP ...|     Turbo|         2.0|        4|              26.0|
| 523986010112|      Aston Martin|            Vanquish| Aspirated|         6.0|       12|              16.0|
| 721554505728|        Volkswagen|                 GTI|     Turbo|         2.0|        4|11.965988038687978|
| 764504178688|    

#### The resulting values found in the cylinders and fuel economy columns are not completely accurate, however, they aare still better than just replacing them with predefined values.


## 6 Handling Outliers in the Dataset:

